# Agri Parcels Analysis

### Introduction
<div style="text-align: justify;">
This study uses the temporal satellite imagery to identify patterns of land use change within the pilot area(Pabbi), such as shifts between agricultural and non-agricultural uses. The expected results include detailed maps and datasets highlighting areas of change, enabling authorities to identifty unauthorized land conversions accordingly. This report presents a concise analysis of a cluster of five **Mauzas** (revenue villages) in Tehsil Pabbi, District Nowshera, Khyber Pakhtunkhwa, Pakistan. The primary objective is to examine land use changes—specifically, the **conversion of agricultural land**—using both cadastral vector datasets and satellite imagery.

The analysis utilizes the original **agricultural parcels (Khasras)** created during the land record digitization process. These parcels were overlaid with **median NDVI values** calculated for the period **January 1, 2024 to December 31, 2024**, derived from Sentinel-2 Harmonized satellite imagery. Based on this NDVI assessment, it is estimated that approximately **19%** of agricultural land has transitioned to non-agricultural use.

---

### Objectives
1. Visualize the cadastral (vector) datasets of the five Mauzas on an interactive map.
2. Display key land use types (e.g., agriculture, built-up areas, roads) in separate layers.
3. Generate a median NDVI layer using **Sentinel-2 Harmonized (10m resolution)** imagery over a one-year period.
4. Identify and visualize agricultural parcels with **median NDVI values between 0.3 and 0.7**, indicative of active cultivation.
5. Quantify the reduction in agricultural land area in **Kanal**.

---

### Datasets

- **Vector Data**  
  The cadastral vector dataset was provided by [Mr. Aftab Ahmad](https://www.linkedin.com/in/aftab-ahmad-4316a463/), Deputy Director GIS, Board of Revenue (BOR), KP. It includes agricultural field boundaries (*Khasras*) categorized as follows:

  | Land Use      | No. of Parcels | Area (Kanal, Approx.) |
  |---------------|----------------|------------------------|
  | Agriculture   | 5,795          | 32,729                 |
  | Stream        | 302            | 822                    |
  | Other         | 229            | 1,047                  |
  | Road/Street   | 172            | 1,668                  |
  | Graveyard     | 29             | 202                    |
  | Built-Up      | 24             | 602                    |

  > **Note:** This dataset is based on the manual land settlement records of 1926. Actual on-ground conditions may have changed since then.

- **Satellite Imagery**  
  - The study uses [Sentinel-2 Harmonized Surface Reflectance](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED) imagery, accessed via Google Earth Engine.
  - Images were filtered by region (five Mauzas), date range (2024), and cloud coverage (<10%).
  - A total of **135 images** were processed to compute NDVI values for the year 2024.

---

## Tools and Libraries Used
- **Google Earth Engine** – Satellite image processing and remote sensing analysis  
- **geemap** – Interactive mapping and Earth Engine integration in Python  
- **GeoPandas** – Spatial data manipulation in Python

---

### Assumptions
- Seasonal analysis (e.g., **Rabi** and **Kharif**) has not been performed. Instead, the study uses a one-year aggregated NDVI analysis to represent changes in agricultural land use.

---

### Findings
- A total of **4,473 agricultural parcels** were identified with **median NDVI values below 0.3**, suggesting possible conversion to non-agricultural uses.
- The total **agricultural area** has declined from **32,729 Kanal** to **26,359 Kanal**, indicating a **19% reduction** in cultivated land since the 1926 settlement.

---
</div>

### Results

![](../images/result.png)
**Figure 1**: An image of the Results.


In [1]:
import ee
import geemap
import geopandas as gpd

In [2]:
geemap.ee.Initialize()

In [3]:
geojson_url = 'https://github.com/code4geoai/gee/releases/download/0.1/pabbigeojson.geojson'

#parquet_file = 'pabbi.parquet'
# The parquet file must be local or can be read from url with request. lets use the local file.


In [4]:
#gdf = gpd.read_parquet(parquet_file)
gdf = gpd.read_file(geojson_url)
gdf['Area_m2'] = gdf['geometry'].area
gdf.head()

,Mouza_Name,Landuse_Ma,Area_Acre,FFID,Parcel_ID,geometry,Area_m2
0,Khushmaqam,Agriculture,0.101785,1,668.0,"MULTIPOLYGON (((753923.077 3769111.141, 753894...",411.907595
1,Khushmaqam,Built up,0.036718,2,670.0,"MULTIPOLYGON (((753959.894 3769126.291, 753959...",148.590279
2,Khushmaqam,Agriculture,0.315557,3,632.0,"MULTIPOLYGON (((753839.267 3769129.693, 753841...",1277.014013
3,Khushmaqam,Agriculture,0.187644,4,669.0,"MULTIPOLYGON (((753952.852 3769131.737, 753946...",759.369193
4,Khushmaqam,Agriculture,0.161568,5,693.0,"MULTIPOLYGON (((754361.202 3769166.424, 754269...",653.842075


In [5]:
# Group the area by Landuse_Ma value
gdf_grouped = gdf.groupby('Landuse_Ma')['Area_m2'].sum().reset_index()
# Now convert square meters to Kanals (1 Kanal = 505.857 m²)
gdf_grouped['Area_Kanal'] = gdf_grouped['Area_m2'] / 505.857
gdf_grouped.round()


,Landuse_Ma,Area_m2,Area_Kanal
0,Agriculture,16556433.0,32729.0
1,Built up,304364.0,602.0
2,Graveyard,101968.0,202.0
3,Other,529528.0,1047.0
4,Road/Streets,843873.0,1668.0
5,Stream,415791.0,822.0


In [6]:

# Convert the GeoDataFrame to an Earth Engine FeatureCollection
pabbi= geemap.gdf_to_ee(gdf)


In [7]:

#Filtering the features having Landuse_Ma as Builtup
builtup=gdf[gdf['Landuse_Ma'] == 'Built up']

builtstyle = {'color': 'white', 
         'width': 1,
         'fillColor': '00000000',
            
         }

In [8]:
#Filtering the features having Landuse_Ma as Agriculture
agri=gdf[gdf['Landuse_Ma'] == 'Agriculture']
agri_feature = geemap.gdf_to_ee(agri)
agristyle = {'color': 'yellow', 
         'width': 1,
         'fillColor': '00000000',
            
         }

In [9]:
#Filtering the features having Landuse_Ma as Graveyard
graveyard=gdf[gdf['Landuse_Ma'] == 'Graveyard']

gravestyle = {'color': 'black', 
         'width': 1,
         'fillColor': '00000000',
            
         }

In [10]:
# Basic Style for Pabbi Vector feature
style = {'color': 'blue', 
         'width': 1,
         'fillColor': '00000000',
             
         }

In [11]:
p = geemap.Map(fullscreen=True)
p.add_basemap('HYBRID')

In [12]:
# Adding Satellite Imagery and calculating NDVI using Sentinel-2
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
# Filter the collection for a specific region and time period
start_date = '2024-01-01'
end_date = '2024-12-31'
# Filter the collection by date and region
filtered_collection = collection.filterDate(start_date, end_date).filterBounds(pabbi.geometry()).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Select the first image from the filtered collection
median_image = filtered_collection.median().clip(pabbi.geometry())

# Calculate NDVI
ndvi = median_image.normalizedDifference(['B8', 'B4']).rename('NDVI')


# Select the bands of interest (e.g.,NVDI)
ndvi_vis = {
    'min': 0.25,
    'max': 0.8,
    'palette': ['white', 'yellow', 'green', 'red']  # Red for highest vegetation
}



# Selecting only those parcels which have ndvi value between .3 and 0.7 

In [13]:
# Step 1: Calculate NDVI for entire area alrady done above

# Step 2: Calculate mean NDVI for each agri polygon
agri_with_ndvi = ndvi.reduceRegions(
    collection=agri_feature,
    reducer=ee.Reducer.mean(),
    scale=10,
)

# Step 3: Filter polygons with NDVI between 0.3 and 0.7
agri_ndvi_filtered = agri_with_ndvi.filter(
    ee.Filter.And(
        ee.Filter.gte('mean', 0.3),
        ee.Filter.lte('mean', 0.7)
    )
)

In [14]:
# Style the filtered pabbi
agrindvi_style = {'color': 'red', 'fillColor': 'FFFF0040'}

In [15]:
# Adding Layers to Map

p.addLayer(ndvi, ndvi_vis,  name='Median NDVI') # Add NDVI layer
p.addLayer(pabbi.style(**style), {}, 'Pabbi Mouzas') # Add Pabbi layer Vector 
p.add_gdf(builtup, layer_name='Built up', style=builtstyle) # Add Builtup layer

p.add_layer(agri_feature.style(**agristyle),{}, name='Agriculture') # Add Agriculture layer

p.add_gdf(graveyard, layer_name='Graveyard', style=gravestyle) # Add Graveyard layer

p.addLayer(agri_ndvi_filtered.style(**agrindvi_style), {}, 'Agri-NDVI (0.3-0.7)Filtered')

p.add_colorbar(
    vis_params=ndvi_vis,
    label='NDVI',
    orientation='horizontal',
    position='bottomright',
    layer_name = 'Median NDVI',
    
)

p.center_object(pabbi, 10)

p

Map(center=[34.02510458074524, 71.77439125564787], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# # Simplify geometry with a tolerance (in meters)
simplified_agri = agri_ndvi_filtered.map(lambda f: f.setGeometry(f.geometry().simplify(30)))
# it didn't helped that much in reducing the time. initially it was 3m.28 seconds and now its also 3m.23s

In [27]:
#Converting the filtered agri polygons to GeoDataFrame

new_gdf = geemap.ee_to_gdf(simplified_agri)
new_gdf.head()

,geometry,Area_Acre,Area_m2,FFID,Landuse_Ma,Mouza_Name,Parcel_ID,mean
0,"LINESTRING (71.74939 34.03201, 71.74868 34.032...",0.315557,1277.014013,3,Agriculture,Khushmaqam,632,0.321833
1,"LINESTRING (71.75503 34.03234, 71.75403 34.032...",0.161568,653.842075,5,Agriculture,Khushmaqam,693,0.502203
2,"POLYGON ((71.75397 34.03242, 71.75401 34.03224...",0.161360,652.999728,7,Agriculture,Khushmaqam,692,0.302640
3,"MULTILINESTRING ((71.7546 34.03248, 71.75436 3...",0.236664,957.747038,8,Agriculture,Khushmaqam,691,0.546135
4,"POLYGON ((71.74868 34.03205, 71.74928 34.03214...",0.624915,2528.941989,9,Agriculture,Khushmaqam,631,0.330838


In [36]:
new_gdf['mean'].min()

np.float64(0.3000960793690901)

In [28]:
# Group the area by Landuse_Ma value
newgdf_grouped = new_gdf.groupby('Landuse_Ma')['Area_m2'].sum().reset_index()
# Now convert square meters to Kanals (1 Kanal = 505.857 m²)
newgdf_grouped['Area_Kanal'] = newgdf_grouped['Area_m2'] / 505.857
newgdf_grouped.round()


,Landuse_Ma,Area_m2,Area_Kanal
0,Agriculture,13333828.0,26359.0


In [29]:
newgdf_grouped.value_counts()

Landuse_Ma   Area_m2       Area_Kanal  
Agriculture  1.333383e+07  26358.888827    1
Name: count, dtype: int64